In [1]:
import pandas as pd
import pgmpy as pgm
import numpy as np
#import mord
from scipy import stats

### Load in data

In [2]:
#mat = pd.read_csv("student-mat.csv", delimiter=';')
#por = pd.read_csv("student-por.csv", delimiter=';')

#mat['subject'] = pd.Series(np.full((mat.shape[0],), fill_value='mat', dtype=object), index=mat.index)
#por['subject'] = pd.Series(np.full((por.shape[0],), fill_value='por', dtype=object), index=por.index)

df = pd.read_csv("matpor_concat.csv", delimiter=',')

In [3]:
df.head()

,Unnamed: 0,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,...,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,subject
0,0,GP,F,18,U,GT3,A,4,4,at_home,...,3,4,1,1,3,6,5,6,6,M
1,1,GP,F,17,U,GT3,T,1,1,at_home,...,3,3,1,1,3,4,5,5,6,M
2,2,GP,F,15,U,LE3,T,1,1,at_home,...,3,2,2,3,3,10,7,8,10,M
3,3,GP,F,15,U,GT3,T,4,2,health,...,2,2,1,1,5,2,15,14,15,M
4,4,GP,F,16,U,GT3,T,3,3,other,...,3,2,1,2,5,4,6,10,10,M


## PgmPy

In [ ]:
def test_independence(df, var1, var2, condition_vars=None):
    """
    Test for the independence condition (var1 _|_ var2 | condition_vars) in df.

    Parameters
    ----------
    df: pandas Dataframe
        The dataset on which to test the independence condition.

    var1: str
        First variable in the independence condition.

    var2: str
        Second variable in the independence condition

    condition_vars: list
        List of variable names in given variables.

    Returns
    -------
    chi_stat: float
        The chi-square statistic for the test.

    p_value: float
        The p-value of the test

    dof: int
        Degrees of Freedom

    Examples
    --------
    >>> df = pd.read_csv('adult.csv')
    >>> chi, p, dof = test_independence(df, var1='Age', var2='Immigrant')
    >>> print("chi =", chi, "\np =", p, "\ndof =",dof)
    chi = 57.7514122288
    p = 8.60514815766e-12
    dof = 4
    >>> chi, p, dof = test_independence(df, var1='Education', var2='HoursPerWeek',
    ...                                 condition_vars=['Age', 'Immigrant', 'Sex'])
    >>> print("chi=", chi, "\np=", p, "\ndof=",dof)
    chi = 1360.65856663 
    p = 0.0 
    dof = 171
    """
    if not condition_vars:
        observed = pd.crosstab(df[var1], df[var2])
        chi_stat, p_value, dof, expected = stats.chi2_contingency(observed)

    else:
        observed_combinations = df.groupby(condition_vars).size().reset_index()
        chi_stat = 0
        dof = 0
        for combination in range(len(observed_combinations)):
            df_conditioned = df.copy()
            for condition_var in condition_vars:
                df_conditioned = df_conditioned.loc[df_conditioned.loc[:, condition_var] == observed_combinations.loc[combination, condition_var]]
            observed = pd.crosstab(df_conditioned[var1], df_conditioned[var2])
            chi, _, freedom, _ = stats.chi2_contingency(observed)
            chi_stat += chi
            dof += freedom
        p_value = 1.0 - stats.chi2.cdf(x=chi_stat, df=dof)

    return chi_stat, p_value, dof

### For defining the network in python, the package pgmpy can be used but it will work only for directed graphs.
from pgmpy.models import BayesianModel

model = BayesianModel([
    ('absences', 'G3'),
    ('activities', 'freetime'),
    ('Age', 'activities'),
    ('Age', 'romantic'),
    ('Age', 'studytime'),
    ('failures', 'G3'),
    ('failures', 'paid'),
    ('failures', 'schoolsup'),
    ('failures', 'studytime'),
    ('famrel', 'famsup'),
    ('famrel', 'paid'),
    ('famsup', 'G3'),
    ('famsup', 'paid'),
    ('famsup', 'studytime'),
    ('Fedu', 'Fjob'),
    ('Fedu', 'higher'),
    ('Fjob', 'famrel'),
    ('freetime', 'G3'),
    ('goout', 'freetime'),
    ('freetime', 'studytime'),
    ('goout', 'romantic'),
    ('health', 'absences'),
    ('health', 'G3'),
    ('health', 'studytime'),
    ('higher', 'famsup'),
    ('higher', 'school'),
    ('higher', 'studytime'),
    ('IQ', 'G3'),
    ('IQ', 'higher'),
    ('IQ', 'studytime'),
    ('Medu', 'higher'),
    ('Medu', 'Mjob'),
    ('Mjob', 'famrel'),
    ('paid', 'freetime'),
    ('paid', 'G3'),
    ('paid', 'studytime'),
    ('romantic', 'freetime'),
    ('school', 'schoolsup'),
    ('school', 'traveltime'),
    ('schoolsup', 'freetime'),
    ('schoolsup', 'G3'),
    ('schoolsup', 'studytime'),
    ('studytime', 'G3'),
    ('subject', 'absences'),
    ('subject', 'failures'),
    ('subject', 'famsup'),
    ('subject', 'G3'),
    ('subject', 'paid'),
    ('subject', 'schoolsup'),
    ('subject', 'studytime'),
    ('traveltime', 'absences'),
    ('traveltime', 'freetime'),
    ('absences', 'failures'),
    # cycles
    # ('studytime', 'activities'),
    # ('activities', 'studytime'),
    # ('freetime', 'goout'),
    # ('failures', 'absences'),
    # ('freetime', 'activities'),
    # ('studytime', 'freetime'),
])

# To test any implied condition in the network, the method `is_active_trail` can be used. Next line tests for 
# the condition (Education _|_ MaritalStatus | Age)
#model.is_active_trail('school', 'schoolsup', observed=['failures'])

# The `get_independencies` method lists all the implied conditions in the model.
model.get_independencies()


# To perform chi-square test on any of the conditional independencies, the method `test_independence` defined
# above can be used. To test for (Education _|_ HoursPerWeek | 'Age', 'Immigrant', 'Sex')
#test_independence(df=df, var1='school', var2='schoolsup', condition_vars=['Age', 'failures', 'paid'])


## Stats

In [ ]:
# all numerical variables
x = [df.age, df.failures, df.absences, df.G1, df.G2, df.G3]

# Ordinal variables
x = [df.Medu, df.Fedu, df.traveltime, df.studytime, df.famrel, df.freetime, df.goout, df.Dalc, df.Walc, df.health, df.absences, df.G1, df.G2, df.G3]


In [ ]:
ord = np.stack(x, axis=1)
rho, pval = stats.spearmanr(ord)

rho.shape

In [ ]:
labels = ["Medu", "Fedu", "traveltime", "studytime", "famrel", "freetme", "goout","Dalc","Walc","health","absences", "G1", "G2", "G3"]

fig, ax = plt.subplots()
im = ax.imshow(rho, cmap='jet')

ax.set_xticks(np.arange(len(labels)))
ax.set_yticks(np.arange(len(labels)))

ax.set_xticklabels(labels)
ax.set_yticklabels(labels)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

ax.set_title("Spearman correlations")

fig.tight_layout()
fig.colorbar(im)

plt.show()